<a href="https://colab.research.google.com/github/broshanfekr/GNN-models/blob/main/GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import math
import torch
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import scipy.sparse as sp
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

Convolution Layer

In [ ]:
class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

Model Definition:

In [ ]:
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

Downloading the dataset

In [ ]:
!pip install gdown
!gdown --id 1406bw-_6zLyNxAuQgWM2kanYpBbl7oqS
!mkdir ./cora
!unzip cora.zip -d ./cora

Loading Dataset

In [ ]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot


def load_data(path="./cora/"):
    """Load citation network dataset"""
    print('Loading cora dataset...')

    idx_features_labels = np.genfromtxt("{}cora.content".format(path), dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}cora.cites".format(path),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test


def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


adj, features, labels, idx_train, idx_val, idx_test = load_data()

Loading cora dataset...


Hyperparameters

In [ ]:
epochs = 200  # Number of epochs to train
lr = 0.01  # Initial learning rate
weight_decay = 5e-4  # Weight decay (L2 loss on parameters)
hidden = 16  # Number of hidden units
dropout = 0.5  # Dropout rate (1 - keep probability)

In [ ]:
# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=hidden,
            nclass=labels.max().item() + 1,
            dropout=dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=lr, weight_decay=weight_decay)

In [ ]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    # Evaluate validation set performance separately,
    # deactivates dropout during validation run.
    model.eval()
    output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

In [ ]:
def mytest():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [ ]:
# Train model
t_total = time.time()
for epoch in range(epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
mytest()

Epoch: 0001 loss_train: 0.3986 acc_train: 0.9429 loss_val: 0.6999 acc_val: 0.8167 time: 0.0242s
Epoch: 0002 loss_train: 0.4011 acc_train: 0.9429 loss_val: 0.7025 acc_val: 0.8167 time: 0.0251s
Epoch: 0003 loss_train: 0.3948 acc_train: 0.9357 loss_val: 0.7059 acc_val: 0.8200 time: 0.0203s
Epoch: 0004 loss_train: 0.4172 acc_train: 0.9143 loss_val: 0.7072 acc_val: 0.8200 time: 0.0198s
Epoch: 0005 loss_train: 0.4073 acc_train: 0.9429 loss_val: 0.7083 acc_val: 0.8200 time: 0.0198s
Epoch: 0006 loss_train: 0.3658 acc_train: 0.9571 loss_val: 0.7059 acc_val: 0.8167 time: 0.0206s
Epoch: 0007 loss_train: 0.3768 acc_train: 0.9643 loss_val: 0.7020 acc_val: 0.8133 time: 0.0219s
Epoch: 0008 loss_train: 0.3639 acc_train: 0.9429 loss_val: 0.6993 acc_val: 0.8133 time: 0.0217s
Epoch: 0009 loss_train: 0.4211 acc_train: 0.9500 loss_val: 0.6984 acc_val: 0.8067 time: 0.0215s
Epoch: 0010 loss_train: 0.3949 acc_train: 0.9429 loss_val: 0.6971 acc_val: 0.8100 time: 0.0220s
Epoch: 0011 loss_train: 0.3732 acc_train